In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pylab as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Tuple
import vectorbt as vbt

plt.rc('text', usetex=True)
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

## Acknowledgement 

Credit to [kridtapon](https://medium.com/@kridtapon) and learning from his posts on Medium.

Volatility is a critical aspect of financial markets, reflecting the degree of variation in the price of a financial instrument over time. Investors and traders use volatility indicators to gauge market uncertainty and identify trading opportunities.

In [2]:
%%capture
ticker = 'HSBA.L'
data = yf.download(ticker, start='2010-01-01', end='2025-01-14')

In [3]:
data_clean = data.copy()
data_clean = data_clean.ffill()
data_clean.columns = [data.columns.values[i][0] for i in range(len(data.columns.values))]
df = data_clean.copy()

In [4]:
df.head()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2010-01-04,333.607025,726.500000,726.500000,713.000000,713.299988,15139314
2010-01-05,338.428619,737.000000,747.200012,723.000000,723.000000,27480532
2010-01-06,339.989960,740.400024,744.000000,734.200012,737.099976,17203196
2010-01-07,338.199066,736.500000,742.400024,730.099976,735.099976,26192632
2010-01-08,339.806366,740.000000,742.500000,729.000000,740.200012,20725091


In [5]:
nrecent = 100
fig = go.Figure(data=[go.Candlestick(
    x=df.index[-nrecent:],
    open=df['Open'][-nrecent:],
    high=df['High'][-nrecent:],
    low=df['Low'][-nrecent:],
    close=df['Close'][-nrecent:]
)])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(
    title=f'{ticker} Stock Price',
    title_x=0.5,  # Center the title
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly_dark'
)
fig.show()

## 1. Chaikin Volatility 

Chaikin Volatility indicator measures the percentage change in the exponential moving average (EMA) of the high-low range over a specified period. This indicator helps identify market volatility changes and potential trend reversals. A high Chaikin Volatility value indicates increased volatility, while a low value suggests decreased volatility.

- High-Low Range 

$$
\text{HL}_{t} = \text{High}_{t} - \text{Low}_{t}
$$

- Exponential Moving Average of High-Low Range

$$
\text{CHV}_{t} = \text{EMA}_{\text{period}} (\text{HL}_{t})
$$

- Chaikin Volatility 

$$
\text{Chaikin Volatility}_{t} = 100\times\left(\dfrac{\text{CHV}_{t} - \text{CHV}_{t-\text{period}}}{\text{CHV}_{t-\text{period}}}\right)
$$

In [7]:

def chaikin_volatility(data: pd.DataFrame, period: int = 10) -> pd.DataFrame:
    """
    Calculate the Chaikin Volatility (CHV) indicator for a given dataset.

    The Chaikin Volatility measures the percentage change in the exponential
    moving average (EMA) of the high-low range over a specified period. It helps
    detect changes in market volatility, indicating potential trend reversals
    or periods of consolidation.

    Args:
        data (pd.DataFrame): A DataFrame containing at least 'High' and 'Low' columns.
        period (int): The period over which to calculate the EMA and percentage change (default is 10).

    Returns:
        pd.DataFrame: The input DataFrame with additional columns:
                      - 'HL': The high-low range.
                      - 'CHV': The EMA of the high-low range.
                      - 'ChaikinVolatility': The percentage change in the EMA.

    Example:
        >>> data = pd.DataFrame({'High': [120, 125, 130], 'Low': [115, 118, 122]})
        >>> chaikin_volatility(data, period=2)
               High  Low     HL        CHV  ChaikinVolatility
        0    120  115   5.0  ... NaN
        1    125  118   7.0  ... NaN
        2    130  122   8.0  ...
    """
    data['HL'] = data['High'] - data['Low']  # High-Low range
    data['CHV'] = data['HL'].ewm(span=period, adjust=False).mean()  # EMA of High-Low range
    data['ChaikinVolatility'] = data['CHV'].pct_change(periods=period) * 100  # % change in EMA
    return data

In [8]:
# Apply the Chaikin Volatility function to the data
df = chaikin_volatility(df)

In [10]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.3)
fig.add_trace(go.Scatter(x=df.index,
                         y=df['CHV'],
                         mode='lines',
                         name='Chaikin Volatility',
                         line=dict(color='orange')),
                         row=1,
                         col=1)
fig.update_layout(
    title=f"Chaikin Volatility Indicator ({ticker})",
    xaxis_title="Date",
    yaxis_title="Chaikin Volatility",
    title_x=0.5,
    template="plotly_dark"
)
fig.show()

In [11]:
def chaikin_volatility_signal(data: pd.DataFrame,
                              increase_threshold: float = 10,
                              decrease_threshold: float = -10) -> pd.Series:
    """
    Generate trading signals based on the Chaikin Volatility (CHV) indicator.

    The function generates signals by identifying sharp increases or decreases
    in the Chaikin Volatility indicator. A significant increase suggests a potential
    breakout, while a significant decrease may indicate a consolidation phase.

    Args:
        data (pd.DataFrame): A DataFrame containing a 'ChaikinVolatility' column.
        increase_threshold (float): Threshold for a sharp increase in volatility
                                    (default is 10%).
        decrease_threshold (float): Threshold for a sharp decrease in volatility
                                    (default is -10%).

    Returns:
        pd.Series: A Series containing trading signals:
                   - 1: Buy signal (sharp increase in volatility).
                   - -1: Sell signal (sharp decrease in volatility).
                   - 0: Hold (no significant change in volatility).

    Example:
        >>> data = pd.DataFrame({'ChaikinVolatility': [0, 12, -15, 8, -5]})
        >>> signals = chaikin_volatility_signal(data)
        >>> signals
        0    0
        1    1
        2   -1
        3    0
        4    0
        dtype: int64
    """
    signals = pd.Series(0, index=data.index)  # Default to hold (0)

    # Generate Buy and Sell signals
    signals[data['ChaikinVolatility'] > increase_threshold] = 1  # Buy signal
    signals[data['ChaikinVolatility'] < decrease_threshold] = -1  # Sell signal

    return signals


## Donchian Channels

The Donchian Channels are a technical analysis indicator used to identify potential breakout levels. They consist of three lines: the upper channel (highest high over a period), the lower channel (lowest low over a period), and the middle channel (average of the upper and lower channels). These channels help traders identify trends and potential reversal points.

- Upper Channel

$$
\text{Upper} = \text{max}(\text{High}_{t-\text{period}+1},\ldots,\text{High}_{t})
$$

- Lower Channel 

$$
\text{Lower} = \text{min}(\text{Low}_{t-\text{period}+1},\ldots,\text{Low}_{t})
$$

- Middle

$$
\text{Middle} = \dfrac{\text{Lower} + \text{Upper}}{2}
$$

In [12]:
def donchian_channels(data: pd.DataFrame, period: int = 20) -> pd.DataFrame:
    """
    Calculate Donchian Channels for a given period.

    Donchian Channels are used to identify potential breakout levels by calculating
    the highest high, the lowest low, and the average of the two over a specified period.

    Args:
        data (pd.DataFrame): A DataFrame with 'High' and 'Low' price columns.
        period (int): The rolling window period for the calculation (default is 20).

    Returns:
        pd.DataFrame: A DataFrame with three new columns:
                      - 'Upper': The upper Donchian channel.
                      - 'Lower': The lower Donchian channel.
                      - 'Middle': The middle Donchian channel.

    Example:
        >>> data = pd.DataFrame({'High': [10, 12, 15, 14], 'Low': [5, 7, 8, 9]})
        >>> donchian_channels(data, period=2)
           High  Low  Upper  Lower  Middle
        0    10    5    NaN    NaN     NaN
        1    12    7   12.0    5.0     8.5
        2    15    8   15.0    7.0    11.0
        3    14    9   15.0    8.0    11.5
    """
    data['Upper'] = data['High'].rolling(window=period).max()
    data['Lower'] = data['Low'].rolling(window=period).min()
    data['Middle'] = (data['Upper'] + data['Lower']) / 2
    return data


In [13]:
df = donchian_channels(df)

# Drop NaN due to rolling window
df = df.dropna()

In [14]:
fig = make_subplots(rows=1, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.3,
                    row_heights=[0.7],
                    row_width=[0.3])

# Plot the Candlestick chart
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'],
                             name="Candlestick",
                             increasing_line_color='green',
                             decreasing_line_color='red'), row=1, col=1)

# Plot the Donchian Channels
fig.add_trace(go.Scatter(x=df.index, y=df['Upper'], mode='lines',
                         name='Upper Channel', line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['Lower'], mode='lines',
                         name='Lower Channel', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['Middle'], mode='lines',
                         name='Middle Channel', line=dict(color='blue', dash='dot')), row=1, col=1)

# Update layout for better presentation
fig.update_layout(
    title=f"Donchian Channels with Candlestick ({ticker})",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    title_x=0.5,
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()

In [15]:
def donchian_channel_signals(data: pd.DataFrame) -> pd.Series:
    """
    Generate trading signals based on Donchian Channels.

    Signals are determined as follows:
    - Buy signal (1): When the 'Close' price crosses above the 'Upper' channel.
    - Sell signal (-1): When the 'Close' price crosses below the 'Lower' channel.
    - Hold signal (0): When the price remains within the channels.

    Args:
        data (pd.DataFrame): A DataFrame containing 'Close', 'Upper', and 'Lower' columns.

    Returns:
        pd.Series: A Series of trading signals (1 = Buy, -1 = Sell, 0 = Hold).

    Example:
        >>> data = pd.DataFrame({
        ...     'Close': [10, 15, 8],
        ...     'Upper': [12, 12, 12],
        ...     'Lower': [7, 7, 7]
        ... })
        >>> donchian_channel_signals(data)
        0    0
        1    1
        2   -1
        dtype: int64
    """
    signals = pd.Series(0, index=data.index)  # Default to hold (0)

    # Generate Buy and Sell signals
    signals[data['Close'] > data['Upper']] = 1  # Buy signal
    signals[data['Close'] < data['Lower']] = -1  # Sell signal

    return signals

## 3. Keltner Channels

Keltner Channels are volatility-based envelopes centered on a moving average. The channels consist of an upper, middle, and lower band, where the middle band is a moving average of the close price, and the upper and lower bands are based on the average true range (ATR) of the price. The formula is as follows

- TR (True Range)

$$
\text{TR} = \text{max}(\text{High}-\text{Low},|\text{High}-\text{Close}|,|\text{Low}-\text{Close}|)
$$

- ATR (Average True Range)

$$
\text{ATR} = \text{Rolling Mean of TR over a period}
$$

- Middle Band (MA of Close)

$$
\text{Middle} = \text{Rolling mean of Close over a period}
$$

- Upper Band

$$
\text{Upper} = \text{Middle} + (\text{ATR}\times \text{Multiplier})
$$

- Lower Band

$$
\text{Lower} = \text{Middle} - (\text{ATR}\times\text{Multiplier})
$$


In [20]:
def keltner_channels(data: pd.DataFrame, period: int = 20, atr_multiplier: float = 2) -> pd.DataFrame:
    """
    Calculate the Keltner Channels for the given data.

    The Keltner Channels are volatility-based envelopes constructed using a moving average (MA) of
    the closing prices and an average true range (ATR) multiplied by a given multiplier to form the
    upper and lower bands.

    Args:
        data (pd.DataFrame): A DataFrame containing columns 'High', 'Low', and 'Close' prices.
        period (int, optional): The period for calculating the moving average and ATR (default is 20).
        atr_multiplier (float, optional): The multiplier for the ATR to calculate the upper and lower bands (default is 2).

    Returns:
        pd.DataFrame: The input data with added columns 'TR', 'ATR', 'Middle', 'Upper', and 'Lower'
                       representing the True Range, Average True Range, Middle Band, Upper Band, and Lower Band, respectively.
    """
    data['TR'] = data[['High', 'Low', 'Close']].apply(lambda x: max(x['High'] - x['Low'],
                                                                    abs(x['High'] - x['Close']),
                                                                    abs(x['Low'] - x['Close'])),
                                                                    axis=1)
    data['ATR'] = data['TR'].rolling(window=period).mean()
    data['Middle'] = data['Close'].rolling(window=period).mean()
    data['Upper'] = data['Middle'] + atr_multiplier * data['ATR']
    data['Lower'] = data['Middle'] - atr_multiplier * data['ATR']
    return data

In [21]:
df = keltner_channels(df)

In [22]:
# Create the subplot figure
fig = make_subplots(
    rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.3,
    row_heights=[0.7],
    row_width=[0.3]
)

# Plot the Candlestick chart
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name="Candlestick",
    increasing_line_color='green',
    decreasing_line_color='red'
    ), row=1, col=1)

# Plot the Keltner Channels
fig.add_trace(go.Scatter(x=df.index, y=df['Upper'], mode='lines',
                         name='Upper Channel', line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['Lower'], mode='lines',
                         name='Lower Channel', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['Middle'], mode='lines',
                         name='Middle Channel', line=dict(color='blue', dash='dot')), row=1, col=1)

# Update layout for better presentation
fig.update_layout(
    title="Keltner Channels with Candlestick (S&P 500)",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    title_x=0.5,
    xaxis_rangeslider_visible=False
)

# Show the plot
fig.show()

In [ ]:
def keltner_signal(data: pd.DataFrame, period: int = 20, atr_multiplier: float = 2) -> pd.Series:
    """
    Generate trading signals based on Keltner Channels.

    Signals are determined as follows:
    - 'Buy' when the price crosses above the upper Keltner channel.
    - 'Sell' when the price crosses below the lower Keltner channel.
    - 'Neutral' when the price is between the upper and lower channels.

    Args:
        data (pd.DataFrame): DataFrame containing 'Close', 'High', and 'Low' columns.
        period (int, optional): Period for calculating moving averages and ATR. Defaults to 20.
        atr_multiplier (float, optional): Multiplier for ATR to calculate upper and lower bands. Defaults to 2.

    Returns:
        pd.Series: A series with 'Buy', 'Sell', or 'Neutral' signals for each row in the data.
    """

    data = keltner_channels(data, period, atr_multiplier)  # Calculate Keltner Channels
    signals = []

    for i in range(len(data)):
        if data['Close'][i] > data['Upper'][i]:
            signals.append('Buy')
        elif data['Close'][i] < data['Lower'][i]:
            signals.append('Sell')
        else:
            signals.append('Neutral')

    return pd.Series(signals, index=data.index)


In [ ]:
# https://medium.com/@kridtapon/python-for-traders-implementing-5-volatility-indicators-to-improve-market-timing-0c090bf73726